# 2. 1 United States - EDA

In [10]:
import pandas as pd
import requests
from io import BytesIO
import numpy as np
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import kaggle

In [11]:
KAGGLE_USERNAME = "marpenalva"
KAGGLE_KEY = "ea42f53179cfa2f6eac83929293413f4"

kaggle.api.authenticate()

def download_file_from_kaggle(dataset, path):
    kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

dataset = "appetukhov/international-trade-database"  # Dataset correcto
download_path = "data/"

if not os.path.exists(download_path):
    os.makedirs(download_path)

download_file_from_kaggle(dataset, download_path)


file_path = os.path.join(download_path, 'trade_1988_2021.csv')
df = pd.read_csv(file_path)

print(df.head())

,ReporterISO3,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeValue in 1000 USD
0,AFG,Afghanistan,SWE,Sweden,2017,Export,86.752
1,AFG,Afghanistan,JOR,Jordan,2018,Export,2796.481
2,AFG,Afghanistan,JOR,Jordan,2017,Export,3100.187
3,AFG,Afghanistan,ITA,Italy,2018,Export,279.918
4,AFG,Afghanistan,ITA,Italy,2017,Export,416.642
...,...,...,...,...,...,...,...
634504,ZWE,Zimbabwe,BRA,Brazil,2000,Export,1267.731
634505,ZWE,Zimbabwe,BOL,Bolivia,2000,Export,2.635
634506,ZWE,Zimbabwe,BMU,Bermuda,2002,Export,10.599
634507,ZWE,Zimbabwe,BLZ,Belize,2000,Export,17.772


In [12]:
df = df.rename(columns={'TradeValue in 1000 USD': 'TradeValue'})

In [13]:
df = df.drop(columns=['ReporterISO3', 'PartnerISO3'])
df = df.rename(columns={'ReporterName': 'Reporter', 'PartnerName': 'Partner', 'TradeFlowName': 'TradeFlow'})


In [14]:
palabra = 'World'
df = df[~df['Partner'].str.contains(palabra)]

## 2. 1. 1 Imports, exports and tarde balance (2000-2020)

In [15]:
usa_imports = df[(df['Partner'] == 'United States') & (df['Year'].between(2000, 2020))]
usa_exports = df[(df['Reporter'] == 'United States') & (df['Year'].between(2000, 2020))]

imports_by_year = usa_imports.groupby('Year')['TradeValue'].sum().reset_index()
exports_by_year = usa_exports.groupby('Year')['TradeValue'].sum().reset_index()

In [16]:
imports_by_year.rename(columns={'TradeValue': 'Imports'}, inplace=True)
exports_by_year.rename(columns={'TradeValue': 'Exports'}, inplace=True)

years = pd.DataFrame({'Year': range(2000, 2021)})

usa_trade = pd.merge(imports_by_year, exports_by_year, on='Year')

usa_trade['Trade Balance'] = usa_trade['Exports'] - usa_trade['Imports']

fig = px.line(usa_trade, x='Year', y=['Imports', 'Exports', 'Trade Balance'], 
              title='Imports, exports and trade balance (2000 - 2020)',
              labels={'value': 'Trade Value in 1000 USD', 'Year': 'Year'},
              markers=True)

fig.update_layout(width=1000, height=600)

pio.write_html(fig, file=f'graphs2/usa/tb.html', auto_open=False)

## 2. 1. 2 Main Export Partners

In [17]:
exporters = usa_exports.groupby('Partner')['TradeValue'].sum().reset_index()

exporters = exporters.sort_values(by='TradeValue', ascending=False)

top_15_exporters = exporters.head(15)

others_value = exporters.iloc[15:]['TradeValue'].sum()

others = pd.DataFrame({'Partner': ['Others'], 'TradeValue': [others_value]})

top_15_exporters = pd.concat([top_15_exporters, others], ignore_index=True)

fig = px.pie(top_15_exporters, 
             values='TradeValue', 
             names='Partner', 
             title='USA: main export partners (2000-2020)',
             labels={'TradeValue': 'Trade Value'})

fig.update_layout(width=800, height=800)

fig.show()

pio.write_html(fig, file=f'graphs2/usa/main.html', auto_open=False)